* Ссылки 
  * [Разведочный анализ данных](https://www.kaggle.com/emstrakhov/eda-with-pandas).<br>

In [ ]:
import os
import csv
import numpy as np
import pandas as pd
import matplotlib 
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline
import seaborn as sns
from functools import reduce

pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_rows = 200
pd.options.display.max_columns = 100

sns.set_style(style='white') 
sns.set(rc={
    'figure.figsize':(12,7), 
    'axes.facecolor': 'white',
    'axes.grid': True, 'grid.color': '.9',
    'axes.linewidth': 1.0,
    'grid.linestyle': u'-'},font_scale=1.5)
custom_colors = ["#3498db", "#95a5a6","#34495e", "#2ecc71", "#e74c3c"]
sns.set_palette(custom_colors)

os.chdir(r'C:\Users\Mr Alex\Documents\GitHub\FlightPreparence')

df = pd.read_csv('../Kaggle/Titanic/train.csv', sep="\t", header = 0, encoding='cp1251', index_col=False)

In [ ]:
#Смотрим коэффициенты корелляций. Мало больших значений - плохо для факторного анализа
df.corr()

In [ ]:
print('Текущая длина фрейма: ', len(df))

In [ ]:
#Удаляем строки с NaN значениями
df = df.loc[~df['data'].isin(['NaN'])]

In [ ]:
#Если несколько классов, но хочется сделать классификацию строго бинарной, то разбиваем на группы ДА и НЕТ
df['date'] = df['date'].replace(0, 1)

In [ ]:
#Преобразуем объектные столбцы в численные. 
df['date'] = df['date'].map({'S': 0, 'C': 1, 'Q': 2})

In [ ]:
#Преобразуем Dtype колонки в bool
df['date'] = df['date'].astype('bool')

In [ ]:
#Преобразуем нечисловые данные в числовые
df['data'] = LabelEncoder().fit_transform(df['data'])

In [ ]:
#Разбиваем колонки на новые, в каждой из которых только свои уникальные значения из прежней колонки
pd.get_dummies(df.Data, prefix="Emb", drop_first = True)

In [ ]:
#Фильтр фрейма, оставить только строки с опредленным значением в одной из колонок
print( 'Before:', len(df) )
gwa_codes = [code for code in df.Code.unique() if 'GWA_' in code]
df = df[df.Code.isin(gwa_codes)]
print( 'After:', len(df) )

In [ ]:
#Удаляем лишние колонки
columns = ['Name', 'Ticket', 'Cabin', 'Fare']
df.drop(columns, inplace=True, axis=1)

In [ ]:
# Удалим первые 10 строк
df = df.drop(np.arange(10), axis=0)

In [ ]:
# Удалим всех пассажиров с переменной меньше 10 и больше 50 
df1 = df.drop(df[(df['data'] < 10) | (df['data'] > 50)].index)
df1.shape[0] / df.shape[0]

In [ ]:
#Заменим все пропущенные значения одной из переменных из колонки
df.Data.fillna(df.Data.mode()[0], inplace = True)

In [ ]:
#Категориальные колонки не имеют естественного порядка, поэтому преобразуем их с помощью one-hot encoding
features = ['predict1', 'predict2', 'predict3']
df = pd.get_dummies(df, columns=features)

In [ ]:
#Преобразуем бинарные столбцы в численные. Колонку y тоже
df['data'] = df['data'].map({'predict1': 0, 'predict2': 1, 'predict3': 2})

In [ ]:
#Приводим множество названий колонок к типу set, находим разность двух множеств: 
print(set(X_train.columns) - set(X_test.columns))
print(set(X_test.columns) - set(X_train.columns))

#Добавляем недостающую колонку. Смотрим, стоит ли склеивать отдельные переменные в более крупные классы
columns = set(X_train.columns) | set(X_test.columns)
X_train = X_train.reindex(columns=columns).fillna(0)
X_test = X_test.reindex(columns=columns).fillna(0)

#Проверяем совпадение колонок (если да, то True)
all(X_train.columns == X_test.columns)

In [ ]:
#Анализ объектов в колонке. Выделение уникальных, сплит по знакам и выделение в отдельную колонку 
df['data_new'] = df.Data.apply(lambda name: name.split(',')[1].split('.')[0].strip()) 
df.Data.nunique()

In [ ]:
#Группировка колонки по двум другим переменным. Медиана группы подставлена в пропущенные строки 
grp = df.groupby(['data1', 'data2'])  
df.Data = grp.Data.apply(lambda x: x.fillna(x.median()))
df.Data.fillna(df.Data.median, inplace = True)

In [ ]:
#Построение облачного графика из объектов, где размер коррелирует с частотой
wc = WordCloud(width = 1000,height = 450,background_color = 'white').generate(str(df.Data_new.values))
plt.imshow(wc, interpolation = 'bilinear')
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

df.Data_new.value_counts()

In [ ]:
#Группируем
grp = df.groupby(['data1', 'data2'])  
df.Data = grp.Data.apply(lambda x: x.fillna(x.median()))

#If still any row remains
td.Age.fillna(td.Age.median, inplace = True)

In [ ]:
#Если в БД нет единой метрики, то стандартизируем данные
import math
from sklearn import preprocessing
from sklearn.decomposition import FactorAnalysis
from sklearn.decomposition import PCA

norm = preprocessing.StandardScaler()
norm.fit(df)
X = norm.transform(df)

#Cтандартизируем переменные 2
df_scaled = preprocessing.scale(df)

#Методом поиска главных компонентов проецируем данные на двумерную плоскость и получаем ранжирование компонентов по важности 
pca = PCA(n_components=5).fit(df_scaled) #Уточняем число компонент и источник данных 

#Доля разброса в данных, объясняемая главными компонентами
print('Влияние компонентов на общий разброс данных: ', pca.explained_variance_ratio_)

In [ ]:
#Повернем фрейм, разбив один столбец на несколько, в зависимости от значений, все объекты выровнены по дате
pivoted_df = df.pivot(index='Date', columns='Code', values='VWAP')
pivoted_df.tail()